$\beta = -1$ means that $\beta$ is selected at random in each iteration.

In [137]:
from enum import IntEnum

class Row(IntEnum):
    BESTS = 0
    AVG_IMP = 1
    AVG_TIME = 2

In [138]:
import pandas as pd

def analyze(dataframe: pd.DataFrame, diffs_slice: slice, times_slice: slice, is_general: bool = False):
    """
    Get averages and #bests of `dataframe` of results from betas experiment.

    `diffs_slice`: `slice` object of the differences columns in `dataframe`.

    `times_slice`: `slice` object of the times columns in `dataframe`.

    `is_general`: `True` if `dataframe` is the table of all instances,
    #bests are counts of minimum averages; otherwise, #bests are counts of 0s.
    """
    diffs = dataframe.iloc[:, diffs_slice]

    other = diffs.min(axis=1) if is_general else 0
    bests = diffs.eq(other, axis=0).sum()

    times = dataframe.iloc[:, times_slice]

    data = {
        Row.BESTS.name: bests,
        Row.AVG_IMP.name: diffs.mean(),
        Row.AVG_TIME.name: times.mean(),
    }

    return pd.DataFrame(data.values(), index=data.keys())

In [139]:
do_save = False
do_display_each = True

In [140]:
from IPython.display import display
from grasp_betas import read_results

BOLD_PROP = "font-weight:bold;"

# data to create dataframe of all instances
datalist = []

names = [
    # "rd400_267_133",
    # "rat783_522_261", # 1
    # "nrw1379_920_459",
    "anpcp_882_441_t5.0",
]
for name in names:
    betas = read_results(name)
    
    if do_display_each:
        print(name)

    # fixed dataframes have the same value for both p and alpha
    for (p, alpha), fixed in betas.groupby("p alpha".split()):
        grouped = fixed.groupby("i p alpha".split(), sort=False).mean()

        # add multiindex
        concated = pd.concat(
            {
                "OF": grouped.iloc[:, :7],
                "diff": grouped.iloc[:, 7:14],
                "time": grouped.iloc[:, 14:21],
                "last_imp": grouped.iloc[:, 21:],
            },
            axis=1,
        )
        
        analysis = analyze(grouped, slice(7, 14), slice(14, 21))
        
        if do_display_each:
            display(
                concated.style
                    # show minimum OF in bold
                    .highlight_min(concated.columns[:7], axis=1, props=BOLD_PROP)
                    # show minimum difference in bold
                    .highlight_min(concated.columns[7:14], axis=1, props=BOLD_PROP)
                    # show minimum time in bold
                    .highlight_min(concated.columns[14:21], axis=1, props=BOLD_PROP)
            )
            display(
                analysis.style
                    # show maximum #bests in bold
                    .highlight_max((Row.BESTS.name, analysis.columns), axis=1, props=BOLD_PROP)
                    .highlight_min((Row.AVG_IMP.name, analysis.columns), axis=1, props=BOLD_PROP)
                    .highlight_min((Row.AVG_TIME.name, analysis.columns), axis=1, props=BOLD_PROP)
            )

        # start row with params
        row = [name, p, alpha]
        # get all average improvements of this combination
        row.extend(analysis.iloc[Row.AVG_IMP, :])
        # get all average times of this combination
        row.extend(analysis.iloc[Row.AVG_TIME, :])
        datalist.append(row)

params = "instance p alpha".split()
# the columns are duplicated because both the differences and the times are included
headers = params + analysis.columns.tolist() * 2
# dataframe of all tests
general = pd.DataFrame(datalist, columns=headers)
# group table so params become indexes of dataframe, instead of columns
gpd_general = general.groupby(params, sort=False).mean()
gpd_concated = pd.concat(
    {
        "diff": gpd_general.iloc[:, :7],
        "time": gpd_general.iloc[:, 7:],
    },
    axis=1,
)

styled_gpd_gen = (
    gpd_concated.style
        .highlight_min(gpd_concated.columns[:7], axis=1, props=BOLD_PROP)
        .highlight_min(gpd_concated.columns[7:], axis=1, props=BOLD_PROP)
)
display(styled_gpd_gen)

analysis = analyze(gpd_general, slice(7), slice(7, 14), is_general=True)
display(
    analysis.style
        .highlight_max((Row.BESTS.name, analysis.columns), axis=1, props=BOLD_PROP)
        .highlight_min((Row.AVG_IMP.name, analysis.columns), axis=1, props=BOLD_PROP)
        .highlight_min((Row.AVG_TIME.name, analysis.columns), axis=1, props=BOLD_PROP)
)

anpcp_882_441_t5.0


,0.0,0.2,0.4,0.6,0.8,1.0,-1
BESTS,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
AVG_IMP,4.459770,5.011494,3.724138,0.000000,8.735632,6.206897,1.747126
AVG_TIME,6.150514,6.431779,8.378463,5.551179,7.024339,7.634817,5.392396


,0.0,0.2,0.4,0.6,0.8,1.0,-1
BESTS,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
AVG_IMP,4.459770,5.011494,3.724138,0.000000,8.735632,6.206897,1.747126
AVG_TIME,6.150514,6.431779,8.378463,5.551179,7.024339,7.634817,5.392396


In [141]:
# styled_gpdtable.format(precision=2).to_latex("styled_gpdt.tex", convert_css=True)

In [142]:
# gpdtable.to_latex("gpdt.tex", float_format="%.2f", multirow=True)